In [1]:
import os

Загружаем модифицированную библиотеку doctr

In [2]:
!git clone https://github.com/something-original/doctr

Cloning into 'doctr'...
remote: Enumerating objects: 12379, done.
remote: Counting objects: 100% (3398/3398), done.
remote: Compressing objects: 100% (969/969), done.
remote: Total 12379 (delta 2856), reused 2568 (delta 2418), pack-reused 8981 (from 1)
Receiving objects: 100% (12379/12379), 6.63 MiB | 7.14 MiB/s, done.
Resolving deltas: 100% (8903/8903), done.


Загружаем датасет с Hugging Face

In [3]:
!wget https://huggingface.co/datasets/DonkeySmall/OCR-Cyrillic-Printed-1/resolve/main/data_1.zip

--2024-12-01 11:30:18--  https://huggingface.co/datasets/DonkeySmall/OCR-Cyrillic-Printed-1/resolve/main/data_1.zip
Resolving huggingface.co (huggingface.co)... 3.167.112.45, 3.167.112.38, 3.167.112.96, ...
Connecting to huggingface.co (huggingface.co)|3.167.112.45|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/8a/2a/8a2af3375f3cdcad634861ea1e25de3fd861e1fea3f8245671179001d11d79dc/162692d384819b68108f5b45c68b32cd9c55737416bfacbc69d82196900b1f9c?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27data_1.zip%3B+filename%3D%22data_1.zip%22%3B&response-content-type=application%2Fzip&Expires=1733311818&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMzMxMTgxOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzhhLzJhLzhhMmFmMzM3NWYzY2RjYWQ2MzQ4NjFlYTFlMjVkZTNmZDg2MWUxZmVhM2Y4MjQ1NjcxMTc5MDAxZDExZDc5ZGMvMTYyNjkyZDM4NDgxOWI2ODEwOGY1YjQ1YzY4YjMyY2Q5YzU1NzM3NDE2YmZhY

Распаковываем и преобразуем в json. Параллельно формируем корпус для синтетического датасета

In [4]:
!unzip -q data_1.zip

In [5]:
import string

In [6]:
print(string.digits)
print(string.punctuation)
print(string.ascii_letters)

0123456789
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ


In [7]:
#русское слово (берем с HF)
#русское cлово + знак препинания (берем с HF и добавляем знак) ".,?!:;"
#русское слово с дефисом внутри слова
#число
#число, разделенное точкой или дефисом (01.01.2024)
#число + символ валюты или процент
#слово или число в скобках, кавычках (с одной стороны или с 2х) "()"«»"
#имя (И.И.)
#№ + числа
#английское слово

In [8]:
punctuation = ".,?!:;"
currency = "$₽"

In [9]:
from tqdm import tqdm
import random

In [10]:
with open('./1.txt', 'r', encoding='utf-8') as labels_txt, \
     open('./labels.json', 'w', encoding='utf-8') as labels_json, \
     open('./corpus.txt', 'w', encoding='utf-8') as corpus:

    have_first_json = False
    labels_json.write('{'+'\n')
    for line in tqdm(labels_txt.readlines()):
        rand = random.randint(1, 30)
        splitted_line = line.split('`')
        img_name = splitted_line[0][2:]
        img_value = splitted_line[1][:-1] if splitted_line[1][-1] == '\n' else splitted_line[1]
        comma = ',' if splitted_line[1][-1] == '\n' else ""

        if rand in range(1, 8): #обычное слово

          if have_first_json:
            labels_json.write(',' + '\n')
            labels_json.write(f"\"{img_name}\": \"{img_value}\"")
          else:
            labels_json.write(f"\"{img_name}\": \"{img_value}\"")
            have_first_json = True

        elif rand in range(9, 12): #слово со знаком препинания
          corpus.write(img_value + random.choice(punctuation) + '\n')

        elif rand in range(13, 15): #слово с дефисом
          index = random.randint(1, len(img_value) - 2)
          corpus.write(img_value[:index] + '-' + img_value[index:] + '\n')

        elif rand in range(16, 18): #число
          corpus.write(''.join(random.choices(string.digits, k=random.randint(1, 12))) + '\n')

        elif rand in range(19, 21): #число с разделителями
          num = ''.join(random.choices(string.digits, k=random.randint(5, 12)))
          n = random.randint(0, 1)
          sep = '.' if n == 0 else '-'
          index1 = random.randint(1, len(num) // 2)
          index2 = random.randint(len(num) // 2 + 1, len(num) - 2)
          corpus.write(''.join(num[:index1]) + sep + ''.join(num[index1:index2]) + sep + ''.join(num[index2:]) + '\n')

        elif rand in range(22, 24): #число + символ валюты, номер или процент
          num = ''.join(random.choices(string.digits, k=random.randint(1, 6)))
          n = random.randint(0, 2)
          if n == 0:
            suffix = '%'
            corpus.write(num + suffix + '\n')
          elif n == 1:
            suffix = random.choice(currency)
            corpus.write(num + suffix + '\n')
          else:
            corpus.write('№' + num + '\n')

        elif rand in range(25, 27): #объект в скобках
          n = random.randint(0, 5)
          if n == 0:
            corpus.write('(' + img_value + '\n')
          elif n == 1:
            corpus.write(img_value + ')' + '\n')
          elif n == 2:
            corpus.write('(' + img_value + ')' + '\n')
          elif n == 3:
            corpus.write('«' + img_value + '\n')
          elif n == 4:
            corpus.write(img_value + '»' + '\n')
          elif n == 5:
            corpus.write('«' + img_value + '»' + '\n')

        elif rand in range(28, 29):
          corpus.write('.'.join([random.choice('АБВГДЕЖЗИКЛМНОПРСТУФХЦЧШЩЭЮЯ') for _ in range(2)]) + '.' + '\n')
        elif rand == 30:
          corpus.write(''.join(random.choices(string.ascii_letters, k=random.randint(1, 15))) + '\n')

    labels_json.write('\n' + '}')

100%|██████████| 500000/500000 [00:03<00:00, 157835.54it/s]


Добавляем синтетический датасет

In [11]:
!pip install pillow==11.0.0

In [22]:
from PIL import Image, ImageDraw, ImageFont

In [13]:
!wget https://legionfonts.com/download/arial-cyr
!unzip -q arial-cyr

--2024-12-01 11:35:30--  https://legionfonts.com/download/arial-cyr
Resolving legionfonts.com (legionfonts.com)... 185.26.99.23
Connecting to legionfonts.com (legionfonts.com)|185.26.99.23|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://legionfonts.com/storage/archives/Arial Cyr.zip [following]
--2024-12-01 11:35:31--  https://legionfonts.com/storage/archives/Arial%20Cyr.zip
Reusing existing connection to legionfonts.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 40564 (40K) [application/zip]
Saving to: ‘arial-cyr’

arial-cyr           100%[===================>]  39.61K  --.-KB/s    in 0s      

2024-12-01 11:35:32 (210 MB/s) - ‘arial-cyr’ saved [40564/40564]



In [26]:
def generate_image_for_word(word, output_directory, counter, font_size):

    word_len = len(word)
    r = random.randint(100, 255)
    g = random.randint(100, 255)
    b = random.randint(100, 255)

    image = Image.new('RGB', (font_size * word_len, font_size + 2), color=(r, g, b))
    draw = ImageDraw.Draw(image)

    try:
        font = ImageFont.truetype("Arial Cyr.ttf", font_size)
    except IOError:
        font = ImageFont.load_default()

    bbox = draw.textbbox((0, 0), text=word, font=font)

    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]

    # Calculate X, Y position of the text to center it
    x = (image.width - text_width) / 2
    y = (image.height - text_height) / 2

    # Draw the text on the image
    draw.text((x, y), word, fill='black', font=font)

    # Save the image
    image_path = os.path.join(output_directory, f"image_2_{counter}.jpg")
    image.save(image_path, 'JPEG')

def generate_images_from_txt(txt_file, output_directory):
    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Read words from the text file
    with open(txt_file, 'r', encoding='utf-8') as file:
        words = file.readlines()

    # Generate an image for each word
    counter = 1
    for word in tqdm(words):
        word = word.strip()  # Remove any leading/trailing whitespace
        if word:  # Check if the line is not empty
            font_size = random.randint(22, 45)
            generate_image_for_word(word, output_directory, counter, font_size)
            counter += 1

In [27]:
# Example usage
txt_file_path = 'corpus.txt'  # Path to your text file containing words
output_dir = 'test'  # Directory where images will be saved

generate_images_from_txt(txt_file_path, output_dir)

100%|██████████| 12/12 [00:00<00:00, 355.87it/s]


In [ ]:
import shutil

In [ ]:
!git clone https://github.com/something-original/synthtiger.git

Cloning into 'synthtiger'...
remote: Enumerating objects: 571, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 571 (delta 11), reused 26 (delta 7), pack-reused 540 (from 1)
Receiving objects: 100% (571/571), 18.84 MiB | 13.55 MiB/s, done.
Resolving deltas: 100% (214/214), done.


Делим на тренировочную, тестовую, валидационную выборки

In [ ]:
import os
import shutil
import re


def split_json_images(json_path, image_folder_path, ranges):

    with open(json_path, encoding='utf-8') as infile:
        lines = infile.readlines()

    for dir in ['train', 'val', 'test']:
        images_dir = f'./{dir}/images'
        if os.path.exists(images_dir):
            shutil.rmtree(images_dir)
        os.makedirs(images_dir)

    for i, (start, end) in enumerate(ranges):

        start_index = start
        end_index = end
        if i == 0:
            path = './train'
        if i == 1:
            path = './val'
        if i == 2:
            path = './test'

        output_file = f'{path}/labels.json'

        with open(output_file, 'w', encoding='utf-8') as outfile:
            if lines[start_index][0] != '{':
                outfile.write('{' + '\n')
            outfile.writelines(lines[start_index:end_index-1])
            if lines[end_index - 1] != '}':
                if lines[end_index - 1][-2] == ',':
                    outfile.write(lines[end_index - 1][:-2] + '\n')
                else:
                    outfile.write(lines[end_index - 1][:-1] + '\n')
                outfile.write('}')
            else:
                outfile.write('}')

        print(f'Written lines {start} to {end} to {output_file}')

        # Copy the specified range of files
        for i in range(start_index, end_index):

            img_filename = lines[i].split(': ')[0].replace("\"",'')

            if img_filename not in ['{\n', '}\n']:
                src_file = os.path.join(image_folder_path, img_filename)
                dest_file = os.path.join(f'{path}/images', img_filename)

                # Copy the file
                shutil.copy2(src_file, dest_file)

Задаем размеры

In [ ]:
input_json_file = 'labels.json'
images_path = './1'
line_ranges = [(0, 10001), (10001, 12501), (12501, 15001)] #train range, val range, test range

In [ ]:
split_json_images(input_json_file, images_path, line_ranges)

Written lines 0 to 10001 to ./train/labels.json
Written lines 10001 to 12501 to ./val/labels.json
Written lines 12501 to 15001 to ./test/labels.json


Запускаем процесс обучения

In [ ]:
os.chdir('doctr')

In [ ]:
!pip install -e . --upgrade
!pip install -r references/requirements.txt
!pip install tf2onnx

Obtaining file:///content/doctr
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for python-doctr (pyproject.toml) ... done
  Created wheel for python-doctr: filename=python_doctr-0.10.1a0-0.editable-py3-none-any.whl size=18127 sha256=d37df366b462abae26c5196677ee3de5384c9974ef871ba8ef9040d7c527e08f
  Stored in directory: /tmp/pip-ephem-wheel-cache-3r9u3quh/wheels/11/6c/0d/cd3d9915ea5e2c63065bac30cfb09530dd9182416ef7bd534a
Successfully built python-doctr
  Attempting uninstall: python-doctr
    Found existing installation: python-doctr 0.10.1a0
    Uninstalling python-doctr-0.10.1a0:
      Successfully uninstalled python-doctr-0.10.1a0
Obtaining file:///content/doctr (from -r references/requirements.txt (line 1))
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... do

In [ ]:
!pip install timm==0.6.12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: timm
    Found existing installation: timm 1.0.11
    Uninstalling timm-1.0.11:
      Successfully uninstalled timm-1.0.11


In [ ]:
!python references/recognition/train_pytorch.py --help

usage: train_pytorch.py [-h] [--train_path TRAIN_PATH] [--val_path VAL_PATH]
                        [--train-samples TRAIN_SAMPLES] [--val-samples VAL_SAMPLES] [--font FONT]
                        [--min-chars MIN_CHARS] [--max-chars MAX_CHARS] [--name NAME]
                        [--epochs EPOCHS] [-b BATCH_SIZE] [--device DEVICE]
                        [--input_size INPUT_SIZE] [--lr LR] [--wd WEIGHT_DECAY] [-j WORKERS]
                        [--resume RESUME] [--vocab VOCAB] [--test-only] [--freeze-backbone]
                        [--show-samples] [--wb] [--push-to-hub] [--pretrained] [--sched SCHED]
                        [--amp] [--find-lr] [--early-stop] [--early-stop-epochs EARLY_STOP_EPOCHS]
                        [--early-stop-delta EARLY_STOP_DELTA]
                        arch

DocTR training script for text recognition (PyTorch)

positional arguments:
  arch                  text-recognition model to train

options:
  -h, --help            show this help message and

Обучение моделей

In [ ]:
!python references/recognition/train_pytorch.py crnn_mobilenet_v3_small --train_path ../train --val_path ../val --vocab russian --epochs 10 --name doctr_crnn_mobilenet_v3_small --freeze-backbone

Streaming output truncated to the last 5000 lines.
Training loss: 3.77694:   1% 2/156 [00:02<01:46,  1.45it/s]
Training loss: 3.77694:   2% 3/156 [00:02<01:26,  1.78it/s]
Training loss: 3.72354:   2% 3/156 [00:02<01:26,  1.78it/s]
Training loss: 3.72354:   3% 4/156 [00:02<00:57,  2.63it/s]
Training loss: 3.80105:   3% 4/156 [00:02<00:57,  2.63it/s]
Training loss: 3.80105:   3% 5/156 [00:02<00:55,  2.70it/s]
Training loss: 3.80633:   3% 5/156 [00:02<00:55,  2.70it/s]
Training loss: 3.80633:   4% 6/156 [00:02<00:42,  3.56it/s]
Training loss: 3.75266:   4% 6/156 [00:02<00:42,  3.56it/s]
Training loss: 3.75266:   4% 7/156 [00:02<00:45,  3.31it/s]
Training loss: 3.71553:   4% 7/156 [00:03<00:45,  3.31it/s]
Training loss: 3.71553:   5% 8/156 [00:03<00:35,  4.19it/s]
Training loss: 3.70205:   5% 8/156 [00:03<00:35,  4.19it/s]
Training loss: 3.70205:   6% 9/156 [00:03<00:38,  3.79it/s]
Training loss: 3.77952:   6% 9/156 [00:03<00:38,  3.79it/s]
Training loss: 3.86523:   6% 9/156 [00:03<00:38, 

In [ ]:
!python references/recognition/train_pytorch.py parseq --train_path ../train --val_path ../val --vocab russian --epochs 10 --name doctr_parseq --freeze-backbone

Namespace(arch='parseq', train_path='../train', val_path='../val', train_samples=1000, val_samples=20, font='FreeMono.ttf,FreeSans.ttf,FreeSerif.ttf', min_chars=1, max_chars=12, name='doctr_parseq', epochs=10, batch_size=64, device=None, input_size=32, lr=0.001, weight_decay=0, workers=None, resume=None, vocab='russian', test_only=False, freeze_backbone=True, show_samples=False, wb=False, push_to_hub=False, pretrained=False, sched='cosine', amp=False, find_lr=False, early_stop=False, early_stop_epochs=5, early_stop_delta=0.01)
Validation set loaded in 0.071s (2500 samples in 40 batches)
Train set loaded in 0.08177s (10000 samples in 156 batches)

  0% 0/156 [00:00<?, ?it/s]
Training loss: 5.1946:   0% 0/156 [00:05<?, ?it/s]
Training loss: 5.1946:   1% 1/156 [00:05<13:54,  5.38s/it]
Training loss: 4.41309:   1% 1/156 [00:05<13:54,  5.38s/it]
Training loss: 4.41309:   1% 2/156 [00:05<06:20,  2.47s/it]
Training loss: 4.14118:   1% 2/156 [00:06<06:20,  2.47s/it]
Training loss: 4.14118:   2

In [ ]:
!python references/recognition/train_pytorch.py master --train_path ../train --val_path ../val --vocab russian --epochs 10 --name doctr_crnn_mobilenet_v3_small --freeze-backbone

Тест моделей

In [ ]:
os.environ["USE_TORCH"] = "1"

In [ ]:
import torch
from doctr.models.predictor import OCRPredictor
from doctr.models.detection.predictor import DetectionPredictor
from doctr.models.recognition.predictor import RecognitionPredictor
from doctr.models.preprocessor import PreProcessor
from doctr.models import fast_base, parseq, master, crnn_mobilenet_v3_small

In [ ]:
detection_model = fast_base(pretrained=True)
recognition_model = parseq(pretrained=False, vocab='russian', num_classes=)

In [ ]:
import string

result = string.ascii_letters
print(len(result))

52


In [ ]:
recognition_params = torch.load("../doctr_parseq.pt", map_location="cpu", weights_only=False)
recognition_model.load_state_dict(recognition_params)

RuntimeError: Error(s) in loading state_dict for PARSeq:
	size mismatch for head.weight: copying a param with shape torch.Size([166, 384]) from checkpoint, the shape in current model is torch.Size([8, 384]).
	size mismatch for head.bias: copying a param with shape torch.Size([166]) from checkpoint, the shape in current model is torch.Size([8]).
	size mismatch for embed.embedding.weight: copying a param with shape torch.Size([168, 384]) from checkpoint, the shape in current model is torch.Size([10, 384]).

In [ ]:
recognition_params['head.weight']

tensor([[-0.0179, -0.0085, -0.0208,  ...,  0.0165,  0.0165, -0.0352],
        [-0.0223, -0.0123,  0.0359,  ..., -0.0058, -0.0516,  0.0091],
        [-0.0180, -0.0218,  0.0224,  ...,  0.0287, -0.0188,  0.0040],
        ...,
        [-0.0464,  0.0095, -0.0757,  ..., -0.0167, -0.0421,  0.0508],
        [-0.0675,  0.0113, -0.0534,  ..., -0.0435, -0.0375,  0.0663],
        [ 0.0066, -0.0149, -0.0111,  ..., -0.0301,  0.0559,  0.0455]])